In [ ]:
import tensorflow as tf
tf.__version__

'2.17.1'

In [ ]:
""" Train model for multiclass sentiment analysis """
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
import tensorflow as tf

# from albert_predict import NUM_LABELS

# Read dataset from CSV
FILE_PATH = 'datasets/beauty_product_data2.csv'
df = pd.read_csv(FILE_PATH)
EPOCH = 20
BATCH = 32
LEARNING_RATE = 1e-5
MAX_LENGTH = 1000
BASE_PRETRAINED_MODEL='albert-base-v2'
NUM_LABELS=3
MODEL_PATH="models/my-product-data-202412221621.h5"

# Prepare data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['reviews'].values,
    df['sentiment'].values,
    test_size=0.2,
    random_state=42
)

# Download ALBERT Pre-trained Model
tokenizer = AlbertTokenizer.from_pretrained(BASE_PRETRAINED_MODEL)
model = TFAlbertForSequenceClassification.from_pretrained(BASE_PRETRAINED_MODEL, num_labels=NUM_LABELS)  # Adjust num_labels based on the number of sentiments



train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')

# Extract NumPy arrays from BatchEncoding
train_input_ids = train_encodings['input_ids'].numpy()
train_attention_mask = train_encodings['attention_mask'].numpy()

test_input_ids = test_encodings['input_ids'].numpy()
test_attention_mask = test_encodings['attention_mask'].numpy()

# Convert sentiment labels to numeric format
label_mapping = {'Positive': 0, 'Neutral': 1, 'Negative': 2}
train_labels_numeric = [label_mapping[label] for label in train_labels]
test_labels_numeric = [label_mapping[label] for label in test_labels]

# Create tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(((train_input_ids, train_attention_mask), train_labels_numeric))
test_dataset = tf.data.Dataset.from_tensor_slices(((test_input_ids, test_attention_mask), test_labels_numeric))

# Training Model
# pylint: disable=no-member
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE) # type: ignore
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) # type: ignore

model.fit(train_dataset.batch(BATCH), epochs=EPOCH) # type: ignore

# Evaluate Model
eval_results = model.evaluate(test_dataset.batch(BATCH)) # type: ignore
print("Test loss:", eval_results[0])
print("Test accuracy:", eval_results[1])

# Predict with the Trained Model
new_texts = ['This serum has worked wonders on my skin. My complexion is noticeably brighter!', 'It’s too early to tell any difference, but I don’t mind the texture.', 'It caused breakouts on my sensitive skin, so I won’t be using it anymore.']
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')

new_input_ids = new_encodings['input_ids'].numpy()
new_attention_mask = new_encodings['attention_mask'].numpy()

predictions = model.predict([new_input_ids, new_attention_mask]) # type: ignore
# Mengambil logits dari TFSequenceClassifierOutput
logits = predictions.logits
# Mengambil prediksi sentimen
predicted_labels = tf.argmax(logits, axis=1).numpy()
predicted_sentiments = [list(label_mapping.keys())[list(label_mapping.values()).index(label)] for label in predicted_labels]
print("Predicted sentiments:", predicted_sentiments)

model.save_weights(MODEL_PATH) # type: ignore

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
10/10 [==============================] - 134s 11s/step - loss: 1.8753 - accuracy: 0.3569
Epoch 2/20
10/10 [==============================] - 114s 11s/step - loss: 1.0888 - accuracy: 0.3771
Epoch 3/20
10/10 [==============================] - 114s 11s/step - loss: 1.0747 - accuracy: 0.4007
Epoch 4/20
10/10 [==============================] - 115s 11s/step - loss: 1.0252 - accuracy: 0.5286
Epoch 5/20
10/10 [==============================] - 114s 11s/step - loss: 0.9124 - accuracy: 0.6128
Epoch 6/20
10/10 [==============================] - 114s 11s/step - loss: 0.7581 - accuracy: 0.7138
Epoch 7/20
10/10 [==============================] - 114s 11s/step - loss: 0.4902 - accuracy: 0.8350
Epoch 8/20
10/10 [==============================] - 114s 11s/step - loss: 0.4359 - accuracy: 0.8889
Epoch 9/20
10/10 [==============================] - 115s 11s/step - loss: 1.3410 - accuracy: 0.6599
Epoch 10/20
10/10 [==============================] - 117s 12s/step - loss: 1.0533 - accuracy: 0.4108

In [ ]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification

# Constants
label_mapping = {'Positive': 0, 'Neutral': 1, 'Negative': 2}
MAX_LENGTH = 1000
NUM_LABELS = 3
MODEL_PATH = 'models/my-product-data-202412221621.h5'
BASE_PRETRAINED_MODEL = 'albert-base-v2'

# Load tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained(BASE_PRETRAINED_MODEL)
model = TFAlbertForSequenceClassification.from_pretrained(BASE_PRETRAINED_MODEL, num_labels=NUM_LABELS)

# Load weights
model.load_weights(MODEL_PATH)

# from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
# import tensorflow as tf

# # Constants
# label_mapping = {'Positive': 0, 'Neutral': 1, 'Negative': 2}
# MAX_LENGTH = 1000
# NUM_LABELS = 3
# MODEL_PATH = 'models/my-product-data-202312181131.h5'
# BASE_PRETRAINED_MODEL = 'albert-base-v2'

# # Load tokenizer and model
# tokenizer = AlbertTokenizer.from_pretrained(BASE_PRETRAINED_MODEL)
# model = TFAlbertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=NUM_LABELS)

# Test texts
new_texts = ["It didn’t help anything.",
            "I didn’t experience any positive results. My skin didn’t feel any smoother or lighter.",
            "This serum really works for me! My complexion is brighter and more even after using it for a few weeks.",
            "After using it for a while, my skin feels the same, no improvement at all.",
            "I noticed a slight improvement, but it’s nothing to get excited about.",
            "The serum is amazing! My skin looks much more even and feels soft.",
            "It caused irritation on my skin, and I didn’t see any benefits.",
            "I have seen subtle improvements in my skin tone, but it's not as fast as I expected.",
            "It gave me a beautiful glow, and my skin is looking radiant now.",
            "This serum made my face oily and did not improve my skin at all."
             ]

# Tokenize
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
new_input_ids = new_encodings['input_ids']
new_attention_mask = new_encodings['attention_mask']

# Predictions
predictions = model.predict({'input_ids': new_input_ids, 'attention_mask': new_attention_mask})
logits = predictions.logits
probabilities = tf.nn.softmax(logits, axis=1).numpy()
predicted_labels = tf.argmax(probabilities, axis=1).numpy()

# Convert labels to sentiments
predicted_sentiments = [list(label_mapping.keys())[list(label_mapping.values()).index(label)] for label in predicted_labels]

print("Predicted sentiments:", predicted_sentiments)

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/1 [==============================] - 5s 5s/step
Predicted sentiments: ['Negative', 'Negative', 'Positive', 'Negative', 'Neutral', 'Positive', 'Negative', 'Negative', 'Positive', 'Negative']
